In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler 

np.random.seed(123456789)
tf.set_random_seed(123456789)

print("start")

## Learning Model
class LearningNetwork:
    # input image size
    nWidth = 512
    nHeight = 512
    nPixels = nWidth * nHeight
    batch_size = 5
    read_threads = 1

    def __init__(self):
        with tf.Graph().as_default():
            self.prepare_model()
            self.prepare_session()
            self.prepare_batch()
    
    def prepare_model(self):
        with tf.name_scope("input"):
            x = tf.placeholder(tf.float32, [None, 512,512,3])
            x_image = tf.cast(tf.reshape(x, [-1, LearningNetwork.nWidth, LearningNetwork.nHeight, 3]), tf.float32)

        
        with tf.name_scope("Convolution_1"):
            # filter size:5×5, input layer:１, output layer:olayer, random initial value in range of stddev0.1
            filter_size1 = 5
            olayer1 = 8
            temp=x_image.get_shape().as_list()[-1]
            W_conv1 = tf.Variable(tf.truncated_normal([filter_size1, filter_size1,temp, olayer1], stddev=0.1))
            h_conv1 = tf.nn.conv2d(x_image, W_conv1, strides=[1,1,1,1], padding='SAME')
            
            
            b_conv1 = tf.Variable(tf.zeros([olayer1]))
            h_conv1_cutoff = tf.nn.relu(h_conv1 + b_conv1)

        with tf.name_scope("Pooling_1"):
            # Pooling layer calculate max values by "max_pool"
            h_pool1 = tf.nn.max_pool(h_conv1_cutoff, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

        with tf.name_scope("Convolution_2"):
            # 2nd Convolution and Pooling layer
            filter_size2 = 5
            olayer2 = 16
            temp=h_pool1.get_shape().as_list()[-1]
            W_conv2 = tf.Variable(tf.truncated_normal([filter_size2, filter_size2,temp, olayer2], stddev=0.1))
            h_conv2 = tf.nn.conv2d(h_pool1, W_conv2, strides=[1,1,1,1], padding='SAME')
            b_conv2 = tf.Variable(tf.zeros([olayer2]))
            h_conv2_cutoff = tf.nn.relu(h_conv2 + b_conv2)

        with tf.name_scope("Pooling_2"):
            h_pool2 = tf.nn.max_pool(h_conv2_cutoff, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
        
        with tf.name_scope("Convolution_3"):
            # 2nd Convolution and Pooling layer
            filter_size3 = 5
            olayer3 = 32
            temp=h_pool2.get_shape().as_list()[-1]
            W_conv3 = tf.Variable(tf.truncated_normal([filter_size3, filter_size3,temp, olayer3], stddev=0.1))
            h_conv3 = tf.nn.conv2d(h_pool2, W_conv3, strides=[1,1,1,1], padding='SAME')
            b_conv3 = tf.Variable(tf.zeros([olayer3]))
            h_conv3_cutoff = tf.nn.relu(h_conv3 + b_conv3)

        with tf.name_scope("Pooling_3"):
            h_pool3 = tf.nn.max_pool(h_conv3_cutoff, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
            

            

        with tf.name_scope("Fully-Connected_1"):
            batch_size,dim1,dim2,dim3=h_pool2.get_shape().as_list()
            newdim=dim1*dim2*dim3
            h_pool3_flat=tf.reshape(h_pool3,[-1,newdim])
            inlayer=h_pool3_flat.shape[1].value
            outlayer = 32

            w1 = tf.Variable(tf.truncated_normal([inlayer]+[outlayer],stddev=0.1))
            b1 = tf.Variable(tf.zeros([outlayer]))
            hidden1=tf.matmul(h_pool3_flat,w1)
            hidden1=tf.add(hidden1,b1)
            hidden1=tf.nn.relu(hidden1)
            

        with tf.name_scope("Fully-Connected_Dropout1"):
            keep_prob = tf.placeholder(tf.float32)
            hidden1_drop = tf.nn.dropout(hidden1, keep_prob)
            
        with tf.name_scope("Fully-Connected_2"):
            inlayer=hidden1_drop.shape[1].value
            outlayer = 32

            w2 = tf.Variable(tf.truncated_normal([inlayer]+[outlayer],stddev=0.1))
            b2 = tf.Variable(tf.zeros([outlayer]))
            hidden2=tf.matmul(hidden1,w2)
            hidden2=tf.add(hidden1,b2)
            hidden2=tf.nn.relu(hidden2)
            

        with tf.name_scope("Fully-Connected_Dropout2"):
            keep_prob = tf.placeholder(tf.float32)
            hidden2_drop = tf.nn.dropout(hidden2, keep_prob)

        with tf.name_scope("Fully-Connected_Output"):
            # Final result of DeepLearning layers
            inlayer=hidden2_drop.shape[1].value
            outlayer = 2
            w3 = tf.Variable(tf.truncated_normal([inlayer]+[outlayer],stddev=0.1))
            b3 = tf.Variable(tf.zeros([outlayer]))
            hidden3=tf.matmul(hidden2_drop,w3)
            hidden3=tf.add(hidden3,b3)
            

            output = hidden3

        with tf.name_scope("Optimizer"):
            ## Define loss(difference between training data and predicted data), and criteria for learning algorithm.
            t = tf.placeholder(tf.int32, [None, 2])
            
            loss=tf.losses.log_loss(labels=t,predictions=output)
            
            train_step = tf.train.AdamOptimizer(0.0001).minimize(loss)
            
        tf.summary.scalar("loss", loss)
        tf.summary.histogram("Fully-Connected_Output:weights", w3)
        tf.summary.histogram("Fully-Connected_Output:biases", b3)
        tf.summary.histogram("Fully-Connected_Hidden:weights", w2)
        tf.summary.histogram("Fully-Connected_Hidden:biases", b2)
        tf.summary.histogram("Convolution_2:weights", W_conv2)
        tf.summary.histogram("Convolution_2:biases", b_conv2)
        tf.summary.histogram("Convolution_1:weights", W_conv1)
        tf.summary.histogram("Convolution_1:biases", b_conv1)
        
        self.x = x
        self.t = t
        self.keep_prob = keep_prob
        self.train_step = train_step
        self.output = output
       
        self.loss = loss
        
    def prepare_session(self):
        sess = tf.Session()
        sess.run(tf.initialize_all_variables())
        saver = tf.train.Saver()
        summary = tf.summary.merge_all()
        writer = tf.summary.FileWriter("board/learn_logs", sess.graph)
        
        self.sess = sess
        self.saver = saver
        self.summary = summary
        self.writer = writer

    # https://www.tensorflow.org/versions/r0.12/how_tos/reading_data/index.html#batching
    def read_my_file_format(self, filename_queue):
        reader = tf.TextLineReader()
        key, record_string = reader.read(filename_queue)
        # "a" means representative value to indicate type for csv cell value.
        image_file_name, label = tf.decode_csv(record_string, [["a"], [1]])

        image_jpeg_data = tf.read_file(image_file_name)
      
        image_decoded = tf.image.decode_jpeg(image_jpeg_data, channels=3)
        image_decoded.set_shape([512, 512, 3])
        
        return image_decoded, label

    def input_pipeline(self, filenames, batch_size, read_threads, num_epochs=None):
        filename_queue = tf.train.string_input_producer(filenames, num_epochs=num_epochs, shuffle=True)
        min_after_dequeue = 3
        capacity = min_after_dequeue + 3 * batch_size

        example_list = [self.read_my_file_format(filename_queue) for _ in range(read_threads)]
        example_batch, label_batch = tf.train.shuffle_batch_join(
            example_list, batch_size=batch_size, capacity=capacity,
            min_after_dequeue=min_after_dequeue)
        return example_batch, label_batch

    def prepare_batch(self):
        
        image_batch, label_batch = self.input_pipeline(['images10.csv'], LearningNetwork.batch_size, LearningNetwork.read_threads)
        
        self.image_batch = image_batch
        self.label_batch = label_batch


nn = LearningNetwork()
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(coord=coord, sess=nn.sess)

i = 0
nLoop = 180000 # 200
print("Start Learning loop")
with nn.sess as sess:
    try:
        while not coord.should_stop():
            i += 1
            
            image_data, label_data = nn.sess.run([nn.image_batch, nn.label_batch])
            image_data = image_data.reshape((LearningNetwork.batch_size, 512,512,3))
            label_data = label_data.reshape((LearningNetwork.batch_size, 2))
            #scaler=MinMaxScaler()
            #image_data = scaler.fit_transform(image_data)
            image_data=image_data/255
            
            nn.sess.run([nn.train_step], feed_dict={nn.x:image_data, nn.t:label_data, nn.keep_prob:0.5})
            if i == nLoop:
                coord.request_stop()

            #TODO:Split data into groups for cross-validation
            if i % 5 == 0:
                loss_vals = []
                loss_val,out, summary = nn.sess.run([nn.loss,nn.output, nn.summary],
                                                            feed_dict={nn.x:image_data, nn.t:label_data, nn.keep_prob:1.0})
                loss_vals.append(loss_val)
                loss_val = np.sum(loss_vals)
                print ('Step: %d, Loss: %f @ %s' % (i, loss_val, datetime.now().strftime("%Y/%m/%d %H:%M:%S")))
                nn.saver.save(nn.sess, 'learn_session', global_step=i)
                
                if i % 5 == 0:
                    
                    nn.writer.add_summary(summary, i)


    except tf.errors.OutOfRangeError:
        print('Done training')
        coord.request_stop(e)
    finally:
        # When done, ask the threads to stop.
        coord.request_stop()

    # stop our queue threads and properly close the session
    coord.request_stop()
    coord.join(threads)

nn.sess.close()

print("end")

start
INFO:tensorflow:Summary name Fully-Connected_Output:weights is illegal; using Fully-Connected_Output_weights instead.
INFO:tensorflow:Summary name Fully-Connected_Output:biases is illegal; using Fully-Connected_Output_biases instead.
INFO:tensorflow:Summary name Fully-Connected_Hidden:weights is illegal; using Fully-Connected_Hidden_weights instead.
INFO:tensorflow:Summary name Fully-Connected_Hidden:biases is illegal; using Fully-Connected_Hidden_biases instead.
INFO:tensorflow:Summary name Convolution_2:weights is illegal; using Convolution_2_weights instead.
INFO:tensorflow:Summary name Convolution_2:biases is illegal; using Convolution_2_biases instead.
INFO:tensorflow:Summary name Convolution_1:weights is illegal; using Convolution_1_weights instead.
INFO:tensorflow:Summary name Convolution_1:biases is illegal; using Convolution_1_biases instead.
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Start Learning loop


ValueError: cannot reshape array of size 5 into shape (5,2)